In [ ]:
!pip install loguru
!pip install pytorch_lightning
!pip install git+https://github.com/huggingface/transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 6.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 800 kB 29.9 MB/s 
     |████████████████████████████████| 125 kB 71.9 MB/s 
     |████████████████████████████████| 512 kB 79.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-oouf1ljj
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-oouf1ljj
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 182 kB 22.0 MB/s 
     |████████████████████████████████| 7.6 MB 71.2 MB/s 
  Created wheel for transformers: filename=

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/Intro_NLP/Code/OCR/macBERT')

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

from transformers import BertTokenizerFast, BertForMaskedLM
from tokenizers import BertWordPieceTokenizer

import operator
import argparse
from abc import ABC
from pathlib import Path
from loguru import logger
from collections import OrderedDict

import lr_scheduler
from evaluate_util import compute_corrector_prf, compute_sentence_level_prf
from base_model import FocalLoss, make_optimizer, build_lr_scheduler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
os.environ["TOKENIZERS_PARALLELISM"] = "FALSE"

In [ ]:
### Config params
class MODEL_class:
    BERT_CKPT= "hfl/chinese-macbert-base"
    DEVICE= "cuda"
    NAME= "macbert4csc"
    GPU_IDS= [0]
    HYPER_PARAMS= [0.3]
    WEIGHTS= ""

class DATASETS_class:
    TRAIN= "/content/gdrive/MyDrive/Intro_NLP/Code/OCR/macBERT/Data/train_mask_can.xlsx"
    VALID= "/content/gdrive/MyDrive/Intro_NLP/Code/OCR/macBERT/Data/test_mask_can.xlsx"
    TEST= "/content/gdrive/MyDrive/Intro_NLP/Code/OCR/macBERT/Data/test_mask_can.xlsx"

class SOLVER_class:
    BASE_LR= 0.00016
    WEIGHT_DECAY= 6e-5
    BATCH_SIZE= 128
    MAX_EPOCHS= 1000
    ACCUMULATE_GRAD_BATCHES= 4
    OPTIMIZER_NAME = "Adam"
    AMSGRAD = True,
    BIAS_LR_FACTOR = 2
    WEIGHT_DECAY_BIAS = 0
    SCHED = "WarmupExponentialLR"
    GAMMA = 0.9999
    STEPS = ()
    WARMUP_FACTOR = 0.005
    WARMUP_EPOCHS = 20
    WARMUP_METHOD = "linear"
    DELAY_ITERS = 0
    ETA_MIN_LR = 3e-7
    MAX_ITER = 100
    WARMUP_ITERS = 5
    INTERVAL = 'step'
    CHECKPOINT_PERIOD = 100
    LOG_PERIOD = 100

class CONFIG:
    MODEL = MODEL_class()
    DATASETS = DATASETS_class()
    SOLVER = SOLVER_class()
    OUTPUT_DIR= "output/macbert4csc_adam",
    MODE= ["train", "test"]


cfg = CONFIG()

In [ ]:
### Prepare data for training tokenizer
os.makedirs("tokenized_data", exist_ok=True)
data_folder ="/content/gdrive/MyDrive/Intro_NLP/Code/OCR/macBERT/Data/"
list_data_4_tokenizer = ["tale-of-kieu-1902.txt", "luc-van-tien-nlvnpf-0059.txt"]
data_4_tokenizer = ""
file_count = 0
for file_name in list_data_4_tokenizer:
  path = data_folder+file_name
  f = open(path, "r")
  content = f.readlines()
  for line in content:
    data_4_tokenizer += line.replace('\n', '\s')
    if len(data_4_tokenizer) >= 4_500:
        with open(f'./tokenized_data/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
                    fp.write('\n'.join(data_4_tokenizer))
        data_4_tokenizer = ""
        file_count += 1
with open(f'./tokenized_data/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
                    fp.write('\n'.join(data_4_tokenizer))
paths = [str(x) for x in Path('./tokenized_data').glob('**/*.txt')]
paths[:5]

['tokenized_data/text_4.txt',
 'tokenized_data/text_2.txt',
 'tokenized_data/text_6.txt',
 'tokenized_data/text_16.txt',
 'tokenized_data/text_12.txt']

In [ ]:
# initialize
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=False,
    lowercase=False
)
# and train
tokenizer.train(files=paths, vocab_size=6_000, min_frequency=0,
                limit_alphabet=6_000, wordpieces_prefix='##',
                special_tokens=[
                    '[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', "[s]", "[/s]"])

os.makedirs(cfg.OUTPUT_DIR[0], exist_ok=True)
tokenizer.save_model(cfg.OUTPUT_DIR[0])

['output/macbert4csc_adam/vocab.txt']

In [ ]:
# Load tokenizer
logger.info(f'load model, model arch: {cfg.MODEL.NAME}')
tokenizer = BertTokenizerFast.from_pretrained(cfg.OUTPUT_DIR[0])

test_string = '新文堂藏板'
enc = tokenizer.encode(test_string)
print(enc)
dec = tokenizer.decode(enc)
print(test_string)
print(dec)

2023-01-02 08:06:52.367 | INFO     | __main__:<module>:2 - load model, model arch: macbert4csc
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[2, 1197, 1185, 622, 2083, 1281, 3]
新文堂藏板
[CLS] 新 文 堂 藏 板 [SEP]


In [ ]:
class DataCollator:
    def __init__(self, tokenizer: BertTokenizerFast):
        self.tokenizer = tokenizer

    def __call__(self, data):
        ori_texts, cor_texts, wrong_idss = zip(*data)
        encoded_texts = [self.tokenizer(t, return_offsets_mapping=True, add_special_tokens=False) for t in ori_texts]
        max_len = max([len(t['input_ids']) for t in encoded_texts]) + 2
        det_labels = torch.zeros(len(ori_texts), max_len).long()

        for i, (encoded_text, wrong_ids) in enumerate(zip(encoded_texts, wrong_idss)):
            off_mapping = encoded_text['offset_mapping']
            for idx in wrong_ids:
                for j, (b, e) in enumerate(off_mapping):
                    if b <= idx < e:
                        # j+1 is because of the previous CLS token
                        det_labels[i, j + 1] = 1
                        break

        return list(ori_texts), list(cor_texts), det_labels

###### Change CscDataset class if the input format is changed       
class CscDataset(Dataset):
    def __init__(self, file_path):
        # self.data = json.load(open(file_path, 'r', encoding='utf-8'))
        self.data = pd.read_excel(file_path, keep_default_na=False)
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # return self.data[index]['original_text'], self.data[index]['correct_text'], self.data[index]['wrong_ids']
        lst_wrong_ids = str(self.data['wrong_ids'].iloc[index])
        if len(lst_wrong_ids) > 0:
          lst_wrong_ids = lst_wrong_ids.split(" ")
          return self.data['original_text'].iloc[index], self.data['correct_text'].iloc[index], [int(x) for x in lst_wrong_ids]
        else:
          return self.data['original_text'].iloc[index], self.data['correct_text'].iloc[index], []


def make_loaders(collate_fn, train_path='', valid_path='', test_path='',
                 batch_size=32, num_workers=4):
    train_loader = None
    if train_path and os.path.exists(train_path):
        train_loader = DataLoader(CscDataset(train_path),
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=num_workers,
                                  collate_fn=collate_fn)
    valid_loader = None
    if valid_path and os.path.exists(valid_path):
        valid_loader = DataLoader(CscDataset(valid_path),
                                  batch_size=batch_size,
                                  num_workers=num_workers,
                                  collate_fn=collate_fn)
    test_loader = None
    if test_path and os.path.exists(test_path):
        test_loader = DataLoader(CscDataset(test_path),
                                 batch_size=batch_size,
                                 num_workers=num_workers,
                                 collate_fn=collate_fn)
    return train_loader, valid_loader, test_loader

In [ ]:
collator = DataCollator(tokenizer=tokenizer)
# Load data
train_loader, valid_loader, test_loader = make_loaders(collator, train_path=cfg.DATASETS.TRAIN,
                                                        valid_path=cfg.DATASETS.VALID, test_path=cfg.DATASETS.TEST,
                                                        batch_size=cfg.SOLVER.BATCH_SIZE, num_workers=4)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
### Base model
class BaseTrainingEngine(pl.LightningModule):
    def __init__(self, cfg, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.cfg = cfg

    def configure_optimizers(self):
        optimizer = make_optimizer(self.cfg, self)
        scheduler = build_lr_scheduler(self.cfg, optimizer)
        return [optimizer], [scheduler]


    def on_validation_epoch_start(self) -> None:
        logger.info('Valid.')

    def on_test_epoch_start(self) -> None:
        logger.info('Testing...')


class CscTrainingModel(BaseTrainingEngine, ABC):
    """
        BaseModel for CSC, defines training and prediction steps
        """

    def __init__(self, cfg, *args, **kwargs):
        super().__init__(cfg, *args, **kwargs)
        # loss weight
        self.w = cfg.MODEL.HYPER_PARAMS[0]

    def training_step(self, batch, batch_idx):
        ori_text, cor_text, det_labels = batch

        # print(f"Original text: {len(ori_text)} - Correct text: {len(cor_text)} - Detection labels: {det_labels.shape}")
        sum_ori_word = 0
        sum_cor_word = 0

        for t in ori_text:
          sum_ori_word += len(t)
        for t in cor_text:
          sum_cor_word += len(t)

        # print(f"Sum original text: {sum_ori_word} - Sum correct text: {sum_cor_word}")
        
        outputs = self.forward(ori_text, cor_text, det_labels)
        loss = self.w * outputs[1] + (1 - self.w) * outputs[0]
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True, batch_size=len(ori_text))
        return loss

    def validation_step(self, batch, batch_idx):
        ori_text, cor_text, det_labels = batch
        outputs = self.forward(ori_text, cor_text, det_labels)
        loss = self.w * outputs[1] + (1 - self.w) * outputs[0]
        det_y_hat = (outputs[2] > 0.5).long()
        cor_y_hat = torch.argmax((outputs[3]), dim=-1)
        encoded_x = self.tokenizer(cor_text, padding=True, return_tensors='pt')
        encoded_x.to(self._device)
        cor_y = encoded_x['input_ids']
        cor_y_hat *= encoded_x['attention_mask']

        results = []
        det_acc_labels = []
        cor_acc_labels = []
        for src, tgt, predict, det_predict, det_label in zip(ori_text, cor_y, cor_y_hat, det_y_hat, det_labels):
            _src = self.tokenizer(src, add_special_tokens=False)['input_ids']
            _tgt = tgt[1:len(_src) + 1].cpu().numpy().tolist()
            _predict = predict[1:len(_src) + 1].cpu().numpy().tolist()
            cor_acc_labels.append(1 if operator.eq(_tgt, _predict) else 0)
            det_acc_labels.append(det_predict[1:len(_src) + 1].equal(det_label[1:len(_src) + 1]))
            results.append((_src, _tgt, _predict,))

        return loss.cpu().item(), det_acc_labels, cor_acc_labels, results

    def validation_epoch_end(self, outputs) -> None:
        det_acc_labels = []
        cor_acc_labels = []
        results = []
        for out in outputs:
            det_acc_labels += out[1]
            cor_acc_labels += out[2]
            results += out[3]
        loss = np.mean([out[0] for out in outputs])
        self.log('val_loss', loss)
        print(f'loss: {loss} - Detection acc: {np.mean(det_acc_labels):.4f} - Correction acc: {np.mean(cor_acc_labels):.4f}')
        # print(f'Detection: acc: {np.mean(det_acc_labels):.4f}')
        # print(f'Correction: acc: {np.mean(cor_acc_labels):.4f}')
        dect_f1, cor_f1 = compute_corrector_prf(results, logger)
        
        _, _, _, senc_f1 = compute_sentence_level_prf(results, logger)
        self.log('val_avg_f1', (dect_f1+cor_f1*2+senc_f1*3)/3)

    def test_step(self, batch, batch_idx):
        return self.validation_step(batch, batch_idx)

    def test_epoch_end(self, outputs) -> None:
        logger.info('Test.')
        self.validation_epoch_end(outputs)

    def predict(self, texts):
        inputs = self.tokenizer(texts, padding=True, return_tensors='pt')
        inputs.to(self.cfg.MODEL.DEVICE)
        with torch.no_grad():
            outputs = self.forward(texts)
            y_hat = torch.argmax(outputs[1], dim=-1)
            expand_text_lens = torch.sum(inputs['attention_mask'], dim=-1) - 1
        rst = []
        for t_len, _y_hat in zip(expand_text_lens, y_hat):
            rst.append(self.tokenizer.decode(_y_hat[1:t_len]).replace(' ', ''))
        return rst


In [ ]:
### MacBert model
class MacBert4Csc(CscTrainingModel, ABC):
    def __init__(self, cfg, tokenizer):
        super().__init__(cfg)
        self.cfg = cfg
        self.bert = BertForMaskedLM.from_pretrained(cfg.MODEL.BERT_CKPT)
        self.detection = nn.Linear(self.bert.config.hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
        self.tokenizer = tokenizer

    def forward(self, texts, cor_labels=None, det_labels=None):
        if cor_labels:
            text_labels = self.tokenizer(cor_labels, padding=True, return_tensors='pt')['input_ids']
            text_labels[text_labels == 0] = -100  # -100 is ignored when calculating 
            text_labels = text_labels.to(self.device)
        else:
            text_labels = None


        encoded_text = self.tokenizer(texts, padding=True, return_tensors='pt')
        encoded_text.to(self.device)

        bert_outputs = self.bert(**encoded_text, labels=text_labels, return_dict=True, output_hidden_states=True)
        # error detection probability
        prob = self.detection(bert_outputs.hidden_states[-1])

        if text_labels is None:
            # error detection output, error correction output
            outputs = (prob, bert_outputs.logits)
        else:
            det_loss_fct = FocalLoss(num_labels=None, activation_type='sigmoid')
            # The pad part does not calculate the loss
            active_loss = encoded_text['attention_mask'].view(-1, prob.shape[1]) == 1
            active_probs = prob.view(-1, prob.shape[1])[active_loss]
            active_labels = det_labels[active_loss]
            det_loss = det_loss_fct(active_probs, active_labels.float())
            # Error detection loss, error correction loss, error detection output, error correction output
            outputs = (det_loss,
                       bert_outputs.loss,
                       self.sigmoid(prob).squeeze(-1),
                       bert_outputs.logits)
        return outputs

In [ ]:
# Initialize model
if cfg.MODEL.NAME == 'macbert4csc':
    model = MacBert4Csc(cfg, tokenizer)
else:
    raise ValueError("model not found.")
    
# Load the previously saved model and continue training
if cfg.MODEL.WEIGHTS and os.path.exists(cfg.MODEL.WEIGHTS):
    model.load_from_checkpoint(checkpoint_path=cfg.MODEL.WEIGHTS, cfg=cfg, map_location=device, tokenizer=tokenizer)

# Configure model save parameters
ckpt_callback = ModelCheckpoint(
    monitor='val_avg_f1',
    dirpath=cfg.OUTPUT_DIR[0],
    filename='{epoch:02d}-{val_loss:.2f}-{val_avg_f1:.3f}',
    save_top_k=1,
    mode='max'
)

# Train the model
logger.info('train model ...')
trainer = pl.Trainer(max_epochs=cfg.SOLVER.MAX_EPOCHS,
                      gpus=None if device == torch.device('cpu') else cfg.MODEL.GPU_IDS,
                      accumulate_grad_batches=cfg.SOLVER.ACCUMULATE_GRAD_BATCHES,
                      callbacks=[ckpt_callback])
# for training
# There is data in train_loader
torch.autograd.set_detect_anomaly(True)
if 'train' in cfg.MODE and train_loader and len(train_loader) > 0:
    if valid_loader and len(valid_loader) > 0:
        trainer.fit(model, train_loader, valid_loader)
    else:
        trainer.fit(model, train_loader)
    logger.info('train model done.')

Downloading:   0%|          | 0.00/659 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at hfl/chinese-macbert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-01-02 08:07:27.733 | INFO     | __main__:<module>:21 - train model ...
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=[0])` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=[0])` instead.
  rank_z

Sanity Checking: 0it [00:00, ?it/s]

2023-01-02 08:07:34.414 | INFO     | __main__:on_validation_epoch_start:14 - Valid.


...

loss: 0.19166488843885335 - Detection acc: 0.6016 - Correction acc: 0.4676
Detection: precision=0.5684296619411123, recall=0.5125368731563422 and F1=0.5390382626680457
Correction: precision=0.7232613908872901, recall=0.6012759170653907 and F1=0.65665142608317
Sentence Level: acc:0.467623, precision:0.569317, recall:0.255608, f1:0.352813


Validation: 0it [00:00, ?it/s]

2023-01-02 14:10:53.153 | INFO     | __main__:on_validation_epoch_start:14 - Valid.


loss: 0.19414015977897428 - Detection acc: 0.6022 - Correction acc: 0.4649
Detection: precision=0.5625163826998689, recall=0.5275319567354966 and F1=0.5444627679817329
Correction: precision=0.7087604846225536, recall=0.5918287937743191 and F1=0.6450381679389312
Sentence Level: acc:0.464933, precision:0.563107, recall:0.256556, f1:0.352507


/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
2023-01-02 14:11:03.898 | INFO     | __main__:<module>:34 - train model done.


In [ ]:
# The model is converted to transformers and can be loaded
if ckpt_callback and len(ckpt_callback.best_model_path) > 0:
    ckpt_path = ckpt_callback.best_model_path
elif cfg.MODEL.WEIGHTS and os.path.exists(cfg.MODEL.WEIGHTS):
    ckpt_path = cfg.MODEL.WEIGHTS
else:
    ckpt_path = ''
logger.info(f'ckpt_path: {ckpt_path}')
if ckpt_path and os.path.exists(ckpt_path):
    model.load_state_dict(torch.load(ckpt_path)['state_dict'])
    # Save the original transformer bert model first
    tokenizer.save_pretrained(cfg.OUTPUT_DIR[0])
    bert = BertForMaskedLM.from_pretrained(cfg.MODEL.BERT_CKPT)
    bert.save_pretrained(cfg.OUTPUT_DIR[0])
    state_dict = torch.load(ckpt_path)['state_dict']
    new_state_dict = OrderedDict()
    if cfg.MODEL.NAME in ['macbert4csc']:
        for k, v in state_dict.items():
            if k.startswith('bert.'):
                new_state_dict[k[5:]] = v
    else:
        new_state_dict = state_dict
    # Save the model file after finetune training and replace the original pytorch_model.bin
    torch.save(new_state_dict, os.path.join(cfg.OUTPUT_DIR[0], 'pytorch_model.bin'))

2023-01-02 14:17:22.380 | INFO     | __main__:<module>:8 - ckpt_path: /content/gdrive/MyDrive/OCR/data/macbert_ckpt/epoch=104-val_loss=0.19-val_avg_f1=0.980.ckpt
Some weights of the model checkpoint at hfl/chinese-macbert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# The logic for testing is the same as training
if 'test' in cfg.MODE and test_loader and len(test_loader) > 0:
    trainer.test(model, test_loader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

2023-01-02 14:17:39.780 | INFO     | __main__:on_test_epoch_start:17 - Testing...
2023-01-02 14:17:45.563 | INFO     | __main__:test_epoch_end:95 - Test.


loss: 0.18866139514879746 - Detection acc: 0.5977 - Correction acc: 0.4714
Detection: precision=0.5713188220230474, recall=0.5484267453294002 and F1=0.5596387808854885
Correction: precision=0.7095472882115643, recall=0.5924401197604791 and F1=0.6457271058535591
Sentence Level: acc:0.471390, precision:0.574658, recall:0.265087, f1:0.362811
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       val_avg_f1           0.9798418283462524
        val_loss            0.18866139514879746
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [ ]:
# from google.colab import files
# !zip -r /content/macbert4csc_adam_2e4.zip /content/output/macbert4csc_adam_2e4
# import shutil
# shutil.move("/content/output/macbert4csc_adam", "/content/gdrive/MyDrive/OCR/data")

In [ ]:
unk_tokens = [' ', '“', '”', '‘', '’', '\n', '…', '—', '\t', '֍']
def get_errors(corrected_text, origin_text):
    sub_details = []
    for i, ori_char in enumerate(origin_text):
        if i >= len(corrected_text):
            break
        if ori_char in unk_tokens:
            # deal with unk word
            corrected_text = corrected_text[:i] + ori_char + corrected_text[i:]
            continue
        if ori_char != corrected_text[i]:
            if ori_char.lower() == corrected_text[i]:
                # pass english upper char
                corrected_text = corrected_text[:i] + ori_char + corrected_text[i + 1:]
                continue
            sub_details.append((ori_char, corrected_text[i], i, i + 1))
    sub_details = sorted(sub_details, key=operator.itemgetter(2))
    return corrected_text, sub_details

    
class Inference:
    def __init__(self, ckpt_path=ckpt_path,
                 vocab_path=cfg.OUTPUT_DIR[0]):
        logger.debug("device: {}".format(device))
        self.tokenizer = BertTokenizerFast.from_pretrained(vocab_path)
        
        self.model = MacBert4Csc.load_from_checkpoint(checkpoint_path=ckpt_path,
                                                          cfg=cfg,
                                                          map_location=device,
                                                          tokenizer=self.tokenizer)
        self.model.to(device)
        self.model.eval()

    def predict(self, sentence_list):
        """
        Text Error Correction Model Prediction
        Args:
            sentence_list: list
                input text list
        Returns: tuple
            corrected_texts(list)
        """
        is_str = False
        if isinstance(sentence_list, str):
            is_str = True
            sentence_list = [sentence_list]
        corrected_texts = self.model.predict(sentence_list)
        if is_str:
            return corrected_texts[0]
        return corrected_texts

    def predict_with_error_detail(self, sentence_list):
        """
        Text error correction model prediction, the result contains error location information
        Args:
            sentence_list: list
                input text list
        Returns: tuple
            corrected_texts(list), details(list)
        """
        details = []
        is_str = False
        if isinstance(sentence_list, str):
            is_str = True
            sentence_list = [sentence_list]
        corrected_texts = self.model.predict(sentence_list)

        for corrected_text, text in zip(corrected_texts, sentence_list):
            corrected_text, sub_details = get_errors(corrected_text, text)
            details.append(sub_details)
        if is_str:
            return corrected_texts[0], details[0]
        return corrected_texts, details

In [ ]:
m = Inference()
inputs = [
        '啓埃元年秋新刋',
        '雲僊古啓新傳',
        '新文堂藏板'
]
answer = [
    '啓定元年秋新刋',
    '雲僊古跡新傳',
    '桞文堂藏板'
]
outputs = m.predict(inputs)
for a, b, c in zip(inputs, answer, outputs):
    print('input  :', a)
    print('answer:', b)
    print('predict:', c)
    print(b==c)
  